In [1]:
import numpy as np
import sys

def generate_solvent_box(solvent_density=0.6, box_size=None, output_file="solvent_box.data"):
    """
    Generate a box of pure solvent particles.
    
    Parameters:
    - solvent_density: solvent density (beads/σ³)
    - box_size: box dimensions [x, y, z] in σ. If None, uses 80% of default gel box
    - output_file: LAMMPS data file name
    """
    
    # Default box size (80% of typical gel box dimensions)
    if box_size is None:
        # Original gel box was ~26 x 26 x 18 (for 8x8x4 units with padding)
        # 20% smaller volume means linear dimensions scaled by (0.8)^(1/3) ≈ 0.928
        scale = 0.8 ** (1/3)
        box_size = [26 * scale, 26 * scale, 18 * scale]
    
    box_x, box_y, box_z = box_size
    total_volume = box_x * box_y * box_z
    num_solvent_total = int(solvent_density * total_volume)
    
    particles = []
    particle_id = 1
    molecule_id = 1
    
    # Add solvent uniformly throughout box
    np.random.seed(42)
    solvent_count = 0
    max_attempts = num_solvent_total * 100
    attempts = 0
    
    while solvent_count < num_solvent_total and attempts < max_attempts:
        attempts += 1
        pos = np.array([
            np.random.rand() * box_x,
            np.random.rand() * box_y,
            np.random.rand() * box_z
        ])
        
        min_dist = 0.8
        too_close = False
        for p in particles[-min(100, len(particles)):]:
            if np.linalg.norm(pos - p['pos']) < min_dist:
                too_close = True
                break
        
        if not too_close:
            particles.append({
                'id': particle_id,
                'type': 1,  # Solvent
                'pos': pos.copy(),
                'mol': molecule_id
            })
            particle_id += 1
            molecule_id += 1
            solvent_count += 1
    
    if solvent_count < num_solvent_total:
        print(f"Warning: Only placed {solvent_count}/{num_solvent_total} solvent particles")
    
    # Write LAMMPS data file
    with open(output_file, 'w') as f:
        f.write("LAMMPS data file for pure solvent box\n\n")
        f.write(f"{len(particles)} atoms\n")
        f.write("0 bonds\n")
        f.write("0 angles\n")
        f.write("0 dihedrals\n")
        f.write("0 impropers\n\n")
        f.write("1 atom types\n\n")
        f.write(f"0.0 {box_x} xlo xhi\n")
        f.write(f"0.0 {box_y} ylo yhi\n")
        f.write(f"0.0 {box_z} zlo zhi\n\n")
        f.write("Masses\n\n")
        f.write("1 1.0  # Solvent\n\n")
        f.write("Atoms\n\n")
        
        for p in particles:
            f.write(f"{p['id']} {p['mol']} {p['type']} {p['pos'][0]:.6f} {p['pos'][1]:.6f} {p['pos'][2]:.6f}\n")
    
    print(f"Generated pure solvent box:")
    print(f"  Box dimensions: {box_x:.2f} x {box_y:.2f} x {box_z:.2f}")
    print(f"  Volume: {total_volume:.2f} σ³")
    print(f"  Target density: {solvent_density:.2f} beads/σ³")
    print(f"  Solvent beads: {solvent_count}")
    print(f"  Actual density: {solvent_count/total_volume:.3f} beads/σ³")
    print(f"  Output: {output_file}")

In [3]:
# Inputs
solvent_density = 1;
box_size = None;
output_file = "solvent_box_rho6_01.data";


generate_solvent_box(solvent_density,box_size,output_file)


Generated pure solvent box:
  Box dimensions: 24.14 x 24.14 x 16.71
  Volume: 9734.40 σ³
  Target density: 1.00 beads/σ³
  Solvent beads: 9734
  Actual density: 1.000 beads/σ³
  Output: solvent_box_rho6_01.data
